In [ ]:
!pip install sentence-transformers qdrant-client

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [ ]:
encoder = SentenceTransformer("intfloat/multilingual-e5-large")

In [ ]:
documents = [
    {
        "text": "Рим е столица на Италия."
    },
    {
        "text": "Rome is the capital city of Italy."
    },
    {
        "text": "Овчарските кучета са добри пазачи."
    }
]

In [ ]:
qdrant = QdrantClient(":memory:")

In [ ]:
qdrant.recreate_collection(
    collection_name="test",
    vectors_config=models.VectorParams(
        # Vector size is defined by used model:
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE,
    )
)

True

In [ ]:
qdrant.upload_records(
    collection_name="test",
    records=[
        models.Record(
            id=idx,
            vector=encoder.encode(doc["text"]).tolist(),
            payload=doc
        )
        for idx, doc in enumerate(documents)
    ]
)

In [ ]:
hits = qdrant.search(
    collection_name="test",
    query_vector=encoder.encode("столица"),
    limit=3,
)

for hit in hits:
    print(hit.payload, "score:", hit.score)

{'text': 'Рим е столица на Италия.'} score: 0.787334680557251
{'text': 'Rome is the capital city of Italy.'} score: 0.7727745771408081
{'text': 'Овчарските кучета са добри пазачи.'} score: 0.7131919860839844
